# Wildlife Strike Analysis

*This notebook should be used to record all of your analysis.*

## Imports

In [1]:
import pandas as pd

from redshift_connector import connect

import altair as alt

from scipy.stats import zscore

### Making a Connection

In [2]:
def get_db_connection():
    return connect(host="c17-redshift-cluster.cdq12ms5gjyk.eu-west-2.redshift.amazonaws.com",
                   database="dw_air_travel",
                   user="najma_hassan",
                   password="Nnajma_71",
                   port=5439)


conn = get_db_connection()
curs = conn.cursor()

**How significant a problem are wildlife strikes?**


1) Get the relevant columns from wildlife_strike table to answer this question and load into pd dataframe

In [73]:
query = """SELECT incident_date ,cost_repairs, indicated_damage, 
nr_injuries,nr_fatalities
FROM wildlife_strike"""
curs.execute(query)
strikes_significance = curs.fetch_dataframe()

2) Clean the data

In [ ]:
strikes_significance['incident_date'] = pd.to_datetime(strikes_significance['incident_date']).dt.year

In [84]:
# Only have rows where year is after 2010
strikes_significance = strikes_significance[strikes_significance['incident_date'] >= 2010]

In [ ]:
def convert_str_to_ints(df, column_name):
    """Replaces empty strings with 0 and converts numeric strings to integers."""
    df[column_name] = df[column_name].replace('', 0)
    df[column_name] = pd.to_numeric(df[column_name]).astype(int)
    return df 
    

In [ ]:
strikes_significance = convert_str_to_ints(strikes_significance, 'nr_injuries')

strikes_significance = convert_str_to_ints(strikes_significance, 'nr_fatalities')

strikes_significance = convert_str_to_ints(strikes_significance, 'cost_repairs')

strikes_significance = convert_str_to_ints(strikes_significance, 'indicated_damage')

In [85]:
strikes_significance.head()

,incident_date,cost_repairs,indicated_damage,nr_injuries,nr_fatalities
87761,2010,0,0,0,0
87762,2010,0,0,0,0
87763,2010,0,0,0,0
87764,2010,0,0,0,0
87765,2010,0,0,0,0


3) Make the Visualisations

_Annual trend chart_

In [93]:
annual_strikes = strikes_significance.groupby('incident_date').size().reset_index(name='Total Incidents')

In [98]:
yearly_trends = alt.Chart(annual_strikes, title = 'Yearly Trend of Wildlife Strikes (2010-2024)').mark_line().encode(
    x = alt.X('incident_date', title='Year of Incident'),
    y = 'Total Incidents',
    tooltip=['incident_date', 'Total Incidents']
)
yearly_trends

alt.Chart(...)

_Cost repairs chart_

In [113]:
strikes_significance['indicated_damage'].value_counts()
damages = pd.DataFrame(columns=['Damage Reported', 'Count'], data=[['Yes', 9303], ['No', 193929]])

In [138]:
strikes_significance[strikes_significance['incident_date'] == 2024 ]['indicated_damage'].value_counts()
damages_2024 = pd.DataFrame(columns=['Damage Reported', 'Count'], data=[
                       ['Yes', 347], ['No', 7310]])

In [129]:
damages_chart = alt.Chart(damages, title = 'Proportion of Reported Damage in Wildlife Strikes').mark_arc().encode(
    theta='Count',
    color= alt.Color('Damage Reported').scale(scheme='bluegreen'),
    tooltip=['Count', 'Damage Reported'],
)
damages_chart

alt.Chart(...)

In [141]:
damages_2024_chart = alt.Chart(damages_2024, title='Proportion of Reported Damage in Wildlife Strikes 2024').mark_arc().encode(
    theta='Count',
    color=alt.Color('Damage Reported').scale(scheme='bluegreen'),
    tooltip=['Count', 'Damage Reported'],
)
damages_2024_chart

alt.Chart(...)

In [149]:
injuries = strikes_significance.groupby('incident_date')['nr_injuries'].sum().reset_index()
fatalities = strikes_significance.groupby(
    'incident_date')['nr_fatalities'].sum().reset_index()

In [179]:
injury_fatalities = injuries.merge(fatalities)
injury_fatalities


,incident_date,nr_injuries,nr_fatalities
0,2010,14,0
1,2011,13,1
2,2012,18,2
3,2013,10,1
4,2014,10,0
5,2015,6,0
6,2016,7,4
7,2017,6,6
8,2018,11,3
9,2019,7,3


In [193]:
injuries_fatalities_chart = alt.Chart(injury_fatalities).transform_fold(['nr_injuries', 'nr_fatalities'], as_=['incident_type', 'count']).mark_bar().encode(
     x=alt.X('incident_date:O', title='Incident Year'),
    y = alt.Y('count:Q', title='Total Injuries and Fatalities'),
    color=alt.Color('incident_type:N').scale(scheme='bluepurple'),
    tooltip=['nr_injuries','nr_fatalities']
)

In [158]:
cost_repairs_by_year = strikes_significance.groupby('incident_date')['cost_repairs'].sum().reset_index()

In [380]:
cost_repairs_chart = alt.Chart(cost_repairs_by_year, title = 'Total Cost Repairs Per year').mark_bar().encode(
    x = alt.X('cost_repairs:Q', title='Cost of Repairs'),
    y= alt.Y('incident_date:O', title='Year'),
    tooltip=['cost_repairs','incident_date'],
    color = alt.Color('cost_repairs').legend(None).scale(scheme='blues')
)

**Are strikes by particular animals more likely/dangerous than others?**


In [200]:
strikes_query = """SELECT species, indicated_damage,cost_repairs, nr_injuries, nr_fatalities 
FROM wildlife_strike"""
curs.execute(strikes_query)
animal_details = curs.fetch_dataframe()

In [202]:
animal_details = convert_str_to_ints(animal_details, 'nr_injuries')

animal_details= convert_str_to_ints(
    animal_details, 'nr_fatalities')

animal_details = convert_str_to_ints(
    animal_details, 'cost_repairs')

animal_details= convert_str_to_ints(
    animal_details, 'indicated_damage')

In [293]:
animal_details['species'].value_counts()

species
Unknown bird - small        50726
Unknown bird - medium       38709
Unknown bird                27860
Mourning dove               15331
Barn swallow                10083
                            ...  
Long-tailed weasel              1
Little owl                      1
Rook                            1
Broad-tailed hummingbird        1
Frigatebirds                    1
Name: count, Length: 916, dtype: int64

In [258]:
top_10 = animal_details.groupby('species').size().sort_values(ascending=False).head(10).reset_index(name='num_strikes')


In [259]:
bottom_10 = animal_details.groupby('species').size().sort_values(
    ascending=False).tail(10).reset_index(name='num_strikes')

In [292]:
animal_num_strikes = pd.concat([top_10, bottom_10])


In [263]:
animal_cost_effect = animal_details.groupby('species')['cost_repairs'].sum().reset_index()

In [291]:
animals_count_and_cost = animal_num_strikes.merge(animal_cost_effect)


In [270]:
animals_chart = alt.Chart(animals_count_and_cost, title ='Strike Frequency and Repair Costs by Top 10 and Bottom 10 Species' ).mark_circle(size=60).encode(
    x= alt.X('num_strikes', title='Number of Strikes'),
    y= alt.Y('cost_repairs', title='Total Cost Repairs'),
    color='species',
    tooltip=['species', 'num_strikes', 'cost_repairs']

)

**When and in what conditions are strikes most likely?**


In [314]:
conditions_query = """SELECT incident_date, time_of_day, sky, precipitation, phase_of_flight, height, speed
FROM wildlife_strike"""
curs.execute(conditions_query)
strike_conditions = curs.fetch_dataframe()

In [316]:
strike_conditions['incident_date'] = pd.to_datetime(
    strike_conditions['incident_date']).dt.year
strike_conditions = strike_conditions[strike_conditions['incident_date'] >= 2010]

In [318]:
strike_conditions = convert_str_to_ints(strike_conditions, 'height')

strike_conditions= convert_str_to_ints(
    strike_conditions, 'speed')

In [304]:
def convert_empty_strings_to_unknown(df, column_name):
    """Replaces empty strings with 'unknown'."""
    df[column_name] = df[column_name].replace('', 'Unknown')
    return df 

In [320]:
strike_conditions = convert_empty_strings_to_unknown(strike_conditions, "time_of_day")
strike_conditions = convert_empty_strings_to_unknown(
    strike_conditions, "sky")
strike_conditions = convert_empty_strings_to_unknown(
    strike_conditions, "precipitation")
strike_conditions = convert_empty_strings_to_unknown(
    strike_conditions, "phase_of_flight")

In [ ]:
sky_strikes = strike_conditions.groupby('sky').size().sort_values(ascending=False).reset_index(name='num_strikes')
# drop row for unknown condition
sky_strikes_excl_ukn = sky_strikes.drop(0)

np.int64(203232)

In [373]:
# percentage of values that are unknown
unknown_sky = sky_strikes[sky_strikes['sky'] == 'Unknown']
((unknown_sky['num_strikes']/203232)*100).round(2)

0    55.92
Name: num_strikes, dtype: float64

In [ ]:
phase_strikes = strike_conditions.groupby(
    'phase_of_flight').size().sort_values(ascending=False).reset_index(name='num_strikes')
# drop row for unknown condition
phase_strikes_excl_ukn = phase_strikes.drop(0)

,phase_of_flight,num_strikes
0,Unknown,88579


In [ ]:
# percentage of values that are unknown
unknown_phase = phase_strikes[phase_strikes['phase_of_flight'] == 'Unknown']
((unknown_phase['num_strikes']/203232)*100).round(2)

0    43.59
Name: num_strikes, dtype: float64

In [ ]:
precipitation_strikes = strike_conditions.groupby('precipitation').size().sort_values(
    ascending=False).reset_index(name='num_strikes')
# drop row for unknown condition
precipitation_strikes_excl_ukn = precipitation_strikes.drop(0)

In [ ]:
# percentage of values that are unknown
unknown_precipitation = precipitation_strikes[precipitation_strikes["precipitation"] == 'Unknown']
((unknown_precipitation['num_strikes']/203232)*100).round(2)

0    96.94
Name: num_strikes, dtype: float64

In [ ]:
time_strikes = strike_conditions.groupby('time_of_day').size().sort_values(
    ascending=False).reset_index(name='num_strikes')
# drop row for unknown condition
time_strikes_excl_ukn = time_strikes.drop(0)

In [ ]:
# percentage of values that are unknown
unknown_time = time_strikes[time_strikes["time_of_day"] == 'Unknown']
((unknown_time['num_strikes']/203232)*100).round(2)

0    47.4
Name: num_strikes, dtype: float64

_Making the charts_

In [392]:
sky_chart = alt.Chart(sky_strikes_excl_ukn).mark_bar().encode(
    x=alt.X('num_strikes:Q', title='Number of Strikes'),
    y = alt.Y('sky:O', title = 'Condition of Sky').sort('-x'),
    tooltip=['sky', 'num_strikes'],
    color= alt.Color('sky').legend(None).scale(scheme='bluegreen', reverse=True)
)
sky_chart

alt.Chart(...)

In [386]:
phase_strikes_excl_ukn

,phase_of_flight,num_strikes
1,Approach,49752
2,Landing Roll,21004
3,Take-off Run,17533
4,Climb,15524
5,En Route,3874
6,Departure,3068
7,Descent,1545
8,Local,1162
9,Arrival,688
10,Taxi,424


In [391]:
phase_of_flight_chart = alt.Chart(phase_strikes_excl_ukn).mark_bar().encode(
    x=alt.X('num_strikes:Q', title='Number of Strikes'),
    y=alt.Y('phase_of_flight:O', title='Phase of Flight').sort('-x'),
    tooltip=['phase_of_flight', 'num_strikes'],
    color=alt.Color('phase_of_flight').legend(None).scale(scheme='bluegreen', reverse=True)
)
phase_of_flight_chart

alt.Chart(...)

In [393]:
precipitation_strikes_excl_ukn

,precipitation,num_strikes
1,Rain,4615
2,Fog,1051
3,Snow,329
4,"Fog, Rain",181
5,"Rain, Snow",18
6,"Fog, Snow",13
7,"None, Rain",3
8,"Fog, Rain, Snow",2


In [395]:
precipitation__chart = alt.Chart(precipitation_strikes_excl_ukn).mark_bar().encode(
    x=alt.X('num_strikes:Q', title='Number of Strikes'),
    y=alt.Y('precipitation:O', title='Precipitation').sort('-x'),
    tooltip=['precipitation', 'num_strikes'],
    color=alt.Color('precipitation').legend(
        None).scale(scheme='bluegreen', reverse=True)
)
precipitation__chart

alt.Chart(...)

In [396]:
time_strikes_excl_ukn

,time_of_day,num_strikes
1,Day,65228
2,Night,33821
3,Dusk,4384
4,Dawn,3462


In [397]:
time_of_day_chart = alt.Chart(time_strikes_excl_ukn).mark_bar().encode(
    x=alt.X('num_strikes:Q', title='Number of Strikes'),
    y=alt.Y('time_of_day:O', title='Time of Day').sort('-x'),
    tooltip=['time_of_day', 'num_strikes'],
    color=alt.Color('time_of_day').legend(
        None).scale(scheme='bluegreen', reverse=True)
)
time_of_day_chart

alt.Chart(...)

**Which airlines/airports/states would be likely potential customers for any of this technology?**


In [422]:
target_query = """SELECT w.airport_name, a.airline_name, w.indicated_damage, w.cost_repairs, w.nr_injuries
FROM wildlife_strike AS w
JOIN airline AS a ON (w.opid = a.iaco)"""
curs.execute(target_query)
target_areas = curs.fetch_dataframe()

In [423]:
target_areas = convert_str_to_ints(
   target_areas, 'indicated_damage')
target_areas = convert_str_to_ints(target_areas, 'nr_injuries')

target_areas = convert_str_to_ints(
  target_areas, 'cost_repairs')

In [424]:
target_areas.head()

,airport_name,airline_name,indicated_damage,cost_repairs,nr_injuries
0,DALLAS/FORT WORTH INTL ARPT,American Airlines,0,0,0
1,PORTLAND INTL (OR),American Airlines,0,0,0
2,LONG BEACH-DAUGH FLD,American Airlines,0,0,0
3,SAN FRANCISCO INTL ARPT,United Airlines,0,0,0
4,DALLAS/FORT WORTH INTL ARPT,American Airlines,0,0,0


In [433]:
airlines_cost = target_areas.groupby(
    'airline_name')['cost_repairs'].sum().sort_values(ascending=False).reset_index().head(5)
airlines_cost

,airline_name,cost_repairs
0,United Airlines,80343451
1,Delta Air Lines,46963162
2,JetBlue,36026984
3,American Airlines,33165588
4,Southwest Airlines,30090092


In [434]:
airport_cost = target_areas.groupby(
    'airport_name')['cost_repairs'].sum().sort_values(ascending=False).reset_index().head(5)
airport_cost

,airport_name,cost_repairs
0,CHICAGO O'HARE INTL ARPT,31662795
1,DENVER INTL AIRPORT,27275552
2,UNKNOWN,19883870
3,GENERAL EDWARD LAWRENCE LOGAN INTL ARPT,15624206
4,SALT LAKE CITY INTL,14769719


In [432]:


airport_chart = alt.Chart(airport_cost).mark_bar().encode(
    x=alt.X('cost_repairs:Q', title='Cost of Repairs'),
    y=alt.Y('airport_name:O', title='Airport Name').sort('-x'),
    tooltip=['airport_name', 'cost_repairs'],
    color=alt.Color('airport_name').legend(
        None).scale(scheme='bluegreen', reverse=True))


airport_chart

alt.Chart(...)

In [438]:
airline_chart = alt.Chart(airlines_cost).mark_bar().encode(
    x=alt.X('cost_repairs:Q', title='Cost of Repairs'),
    y=alt.Y('airline_name:O', title='Airline Name').sort('-x'),
    tooltip=['airline_name', 'cost_repairs'],
    color=alt.Color('airline_name').legend(
        None).scale(scheme='bluegreen', reverse=True))


airline_chart

alt.Chart(...)

In [416]:
target_areas['airline_name'].unique()

array(['American Airlines', 'United Airlines', 'Delta Air Lines',
       'Southwest Airlines', 'Eastern Airlines', 'Hawaiian Airlines',
       'Sun Country Airlines', 'Frontier Airlines', 'Spirit Airlines',
       'Allegiant Air', 'JetBlue', 'Breeze Airways', 'Avelo Airlines'],
      dtype=object)